Python program for creating TestBucket(S3 bucket) and S3-object-size-history(dynamodb table)

In [9]:
import boto3
region_name = 'us-east-1'

In [10]:
def create_s3_bucket(bucket_name):
    s3 = boto3.client('s3',region_name=region_name)
    try:
        s3.create_bucket(Bucket=bucket_name)
        print(f'Bucket {bucket_name} created successfully.')
    except Exception as e:
        print(f'Error creating bucket: {e}')

In [11]:
def create_dynamodb_table(table_name):
    dynamodb = boto3.client('dynamodb',region_name=region_name)
    try:
        response = dynamodb.create_table(
            TableName=table_name,
            KeySchema=[
                {
                    'AttributeName': 'BucketName',
                    'KeyType': 'HASH'  # Partition key
                },
                {
                    'AttributeName': 'Timestamp',
                    'KeyType': 'RANGE'  # Sort key
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'BucketName',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'Timestamp',
                    'AttributeType': 'N'
                }
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        )
        print(f'Table {table_name} created successfully.')
    except Exception as e:
        print(f'Error creating table: {e}')

In [7]:
create_s3_bucket('testbucketkiita0130')
create_dynamodb_table('S3-object-size-history')

Bucket testbucketkiita0130 created successfully.
Table S3-object-size-history created successfully.


In [14]:
def add_gsi_to_table(table_name):
    dynamodb = boto3.client('dynamodb', region_name=region_name)
    try:
        response = dynamodb.update_table(
            TableName=table_name,
            AttributeDefinitions=[
                {
                    'AttributeName': 'BucketName',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'TotalSize',
                    'AttributeType': 'N'
                }
            ],
            GlobalSecondaryIndexUpdates=[
                {
                    'Create': {
                        'IndexName': 'BucketName-TotalSize-index',
                        'KeySchema': [
                            {
                                'AttributeName': 'BucketName',
                                'KeyType': 'HASH'
                            },
                            {
                                'AttributeName': 'TotalSize',
                                'KeyType': 'RANGE'
                            }
                        ],
                        'Projection': {
                            'ProjectionType': 'ALL'
                        },
                        'ProvisionedThroughput': {
                            'ReadCapacityUnits': 5,
                            'WriteCapacityUnits': 5
                        }
                    }
                }
            ]
        )
        print(f'GSI added successfully to table {table_name}.')
    except Exception as e:
        print(f'Error updating table: {e}')

In [15]:
add_gsi_to_table('S3-object-size-history')

GSI added successfully to table S3-object-size-history.
